In [3]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
import joblib
import yfinance as yf
from datetime import datetime

In [5]:
ticker = 'NVDA'
start_date = '2020-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')

In [8]:
data = yf.download(ticker, start=start_date, end=end_date)

# Show all features
data.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,5.96875,5.99775,5.91800,5.99775,5.973633,237536000
2020-01-03,5.87750,5.94575,5.85250,5.90175,5.878019,205384000
2020-01-06,5.80800,5.93175,5.78175,5.92650,5.902669,262636000
2020-01-07,5.95500,6.04425,5.90975,5.99825,5.974132,314856000
2020-01-08,5.99400,6.05100,5.95375,6.00950,5.985337,277108000


In [9]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Close Price'))
fig.update_layout(title=f'{ticker} Closing Price', xaxis_title='Date', yaxis_title='Price')
fig.show()

In [11]:
n_steps = 10
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

In [12]:
X, y = [], []
for i in range(len(scaled_data) - n_steps):
    X.append(scaled_data[i:i+n_steps])
    y.append(scaled_data[i+n_steps])
X, y = np.array(X), np.array(y)

In [14]:
split = int(len(X) * 0.8)
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

In [26]:
# Build and train LSTM model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps, 1), return_sequences=True))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [27]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [28]:
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2, verbose=0, callbacks=[early_stopping])

In [29]:
# Plot training history
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(history.history['loss'])), y=history.history['loss'], mode='lines', name='Training Loss'))
fig.add_trace(go.Scatter(x=np.arange(len(history.history['val_loss'])), y=history.history['val_loss'], mode='lines', name='Validation Loss'))
fig.update_layout(title='Model Loss Over Epochs', xaxis_title='Epochs', yaxis_title='Loss')
fig.show()

In [30]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(history.history['mae'])), y=history.history['mae'], mode='lines', name='Training MAE'))
fig.add_trace(go.Scatter(x=np.arange(len(history.history['val_mae'])), y=history.history['val_mae'], mode='lines', name='Validation MAE'))
fig.update_layout(title='Model MAE Over Epochs', xaxis_title='Epochs', yaxis_title='Mean Absolute Error (MAE)')
fig.show()

In [31]:
forecast_periods = 2
forecast = []
last_sequence = X[-1]
for _ in range(forecast_periods):
    pred = model.predict(last_sequence.reshape((1, n_steps, 1)))
    forecast.append(pred[0, 0])
    last_sequence = np.append(last_sequence[1:], pred[0, 0])
forecast = scaler.inverse_transform(np.array(forecast).reshape(-1, 1))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 712ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step


In [33]:
# Plot actual vs predicted
fig = go.Figure()

# Plot actual values
fig.add_trace(go.Scatter(
    x=data.index[-len(y_test):],
    y=scaler.inverse_transform(y_test).flatten(),
    mode='lines',
    name='Actual'
))

# Plot predicted values
forecast_dates = pd.date_range(start=data.index[-1] + pd.Timedelta(days=1), periods=forecast_periods, freq='B')
fig.add_trace(go.Scatter(
    x=forecast_dates,
    y=forecast.flatten(),
    mode='markers+lines',  # Scatter and lines
    name='Predicted'
))

fig.update_layout(
    title='Actual vs Predicted Prices',
    xaxis_title='Date',
    yaxis_title='Price'
)

fig.show()


In [34]:
joblib.dump(model, 'lstm_model.pkl')

['lstm_model.pkl']